# 第 8 章 数据规整：聚合、合并和重塑

## 8.1 层次化索引

In [6]:
data= pd.Series(np.random.randn(9)
               , index=[list('aaabbccdd')
                       , [1,2,3,1,3,1,2,2,3]])

In [4]:
import pandas as pd

In [5]:
import numpy as np

In [7]:
data

a  1   -0.578781
   2    0.021189
   3   -1.405129
b  1    1.162735
   3   -1.907007
c  1    0.468289
   2   -0.552469
d  2   -0.840247
   3   -0.155247
dtype: float64

In [8]:
data.index

MultiIndex(levels=[['a', 'b', 'c', 'd'], [1, 2, 3]],
           codes=[[0, 0, 0, 1, 1, 2, 2, 3, 3], [0, 1, 2, 0, 2, 0, 1, 1, 2]])

In [9]:
data['b':'c']

b  1    1.162735
   3   -1.907007
c  1    0.468289
   2   -0.552469
dtype: float64

In [10]:
data.loc[:,2]

a    0.021189
c   -0.552469
d   -0.840247
dtype: float64

In [11]:
data.unstack()

,1,2,3
a,-0.578781,0.021189,-1.405129
b,1.162735,NaN,-1.907007
c,0.468289,-0.552469,NaN
d,NaN,-0.840247,-0.155247


In [12]:
data

a  1   -0.578781
   2    0.021189
   3   -1.405129
b  1    1.162735
   3   -1.907007
c  1    0.468289
   2   -0.552469
d  2   -0.840247
   3   -0.155247
dtype: float64

In [13]:
data.unstack().stack()

a  1   -0.578781
   2    0.021189
   3   -1.405129
b  1    1.162735
   3   -1.907007
c  1    0.468289
   2   -0.552469
d  2   -0.840247
   3   -0.155247
dtype: float64

In [15]:
frame=pd.DataFrame(np.arange(12).reshape((4,3))
                  , index=[list('aabb')
                          , [1,2,1,2]]
                  , columns=[['Ohio','Ohio','Colorado']
                            , ['Green','Red','Green']])

In [16]:
frame

Ohio     Colorado
    Green Red    Green
a 1     0   1        2
  2     3   4        5
b 1     6   7        8
  2     9  10       11

In [17]:
frame.index.names=['key1','key2']

In [18]:
frame.columns.names=['state','color']

In [19]:
frame

state      Ohio     Colorado
color     Green Red    Green
key1 key2                   
a    1        0   1        2
     2        3   4        5
b    1        6   7        8
     2        9  10       11

In [20]:
frame['Ohio']

color      Green  Red
key1 key2            
a    1         0    1
     2         3    4
b    1         6    7
     2         9   10

### 重排与分级排序

In [22]:
frame

state      Ohio     Colorado
color     Green Red    Green
key1 key2                   
a    1        0   1        2
     2        3   4        5
b    1        6   7        8
     2        9  10       11

In [23]:
frame.swaplevel('key1','key2')

state      Ohio     Colorado
color     Green Red    Green
key2 key1                   
1    a        0   1        2
2    a        3   4        5
1    b        6   7        8
2    b        9  10       11

In [24]:
frame.sort_index(level=1)

state      Ohio     Colorado
color     Green Red    Green
key1 key2                   
a    1        0   1        2
b    1        6   7        8
a    2        3   4        5
b    2        9  10       11

In [26]:
frame.swaplevel(0,1).sort_index(level=0)

state      Ohio     Colorado
color     Green Red    Green
key2 key1                   
1    a        0   1        2
     b        6   7        8
2    a        3   4        5
     b        9  10       11

### 根据级别汇总统计

In [27]:
frame.sum(level='key2')

state  Ohio     Colorado
color Green Red    Green
key2                    
1         6   8       10
2        12  14       16

In [28]:
frame.sum(level='color',axis=1)

color      Green  Red
key1 key2            
a    1         2    1
     2         8    4
b    1        14    7
     2        20   10

### 使用DataFrame的列进行索引

In [30]:
frame=pd.DataFrame({'a':range(7)
                   ,'b':range(7,0,-1)
                   ,'c':['one','one','one','two','two','two','two']
                   ,'d':[0,1,2,0,1,2,3]})

In [31]:
frame

,a,b,c,d
0,0,7,one,0
1,1,6,one,1
2,2,5,one,2
3,3,4,two,0
4,4,3,two,1
5,5,2,two,2
6,6,1,two,3


In [32]:
frame2=frame.set_index(['c','d'])

In [33]:
frame2

a  b
c   d      
one 0  0  7
    1  1  6
    2  2  5
two 0  3  4
    1  4  3
    2  5  2
    3  6  1

In [34]:
frame.set_index(['c','d']
               , drop=False)

a  b    c  d
c   d              
one 0  0  7  one  0
    1  1  6  one  1
    2  2  5  one  2
two 0  3  4  two  0
    1  4  3  two  1
    2  5  2  two  2
    3  6  1  two  3

In [35]:
frame2.reset_index()

,c,d,a,b
0,one,0,0,7
1,one,1,1,6
2,one,2,2,5
3,two,0,3,4
4,two,1,4,3
5,two,2,5,2
6,two,3,6,1


## 8.2 合并数据集

### 数据库风格的DataFrame合并

In [43]:
df1=pd.DataFrame({'key':list('bbacaab')
                 ,'data1':range(7)})

In [44]:
df1

,key,data1
0,b,0
1,b,1
2,a,2
3,c,3
4,a,4
5,a,5
6,b,6


In [45]:
df2=pd.DataFrame({'key':list('abd')
                 ,'dat2':range(3)})

In [46]:
df2

,key,dat2
0,a,0
1,b,1
2,d,2


In [47]:
pd.merge(df1,df2)

,key,data1,dat2
0,b,0,1
1,b,1,1
2,b,6,1
3,a,2,0
4,a,4,0
5,a,5,0


In [48]:
# merge默认将重叠列的列名当做键

In [49]:
pd.merge(df1,df2,on='key')

,key,data1,dat2
0,b,0,1
1,b,1,1
2,b,6,1
3,a,2,0
4,a,4,0
5,a,5,0


In [57]:
df3=pd.DataFrame({'lkey':list('bbacaab')
                 ,'data1':range(7)})

In [58]:
df4=pd.DataFrame({'rkey':list('abd')
                 ,'data2':range(3)})

In [52]:
pd.merge(df3,df4,left_on='lkey',right_on='rkey')

,lkey,rkey_x,rkey_y,data2
0,b,0,b,1
1,b,1,b,1
2,b,6,b,1
3,a,2,a,0
4,a,4,a,0
5,a,5,a,0


In [53]:
pd.merge(df1,df2,how='outer')

,key,data1,dat2
0,b,0.0,1.0
1,b,1.0,1.0
2,b,6.0,1.0
3,a,2.0,0.0
4,a,4.0,0.0
5,a,5.0,0.0
6,c,3.0,NaN
7,d,NaN,2.0


In [54]:
pd.merge(df3,df4,left_on='lkey',right_on='rkey',how='left')

,lkey,rkey_x,rkey_y,data2
0,b,0,b,1.0
1,b,1,b,1.0
2,a,2,a,0.0
3,c,3,NaN,NaN
4,a,4,a,0.0
5,a,5,a,0.0
6,b,6,b,1.0


In [55]:
pd.merge(df3,df4,left_on='lkey',right_on='rkey',how='right')

,lkey,rkey_x,rkey_y,data2
0,b,0.0,b,1
1,b,1.0,b,1
2,b,6.0,b,1
3,a,2.0,a,0
4,a,4.0,a,0
5,a,5.0,a,0
6,NaN,NaN,d,2


In [59]:
pd.merge(df3,df4,left_on='lkey',right_on='rkey',how='outer')

,lkey,data1,rkey,data2
0,b,0.0,b,1.0
1,b,1.0,b,1.0
2,b,6.0,b,1.0
3,a,2.0,a,0.0
4,a,4.0,a,0.0
5,a,5.0,a,0.0
6,c,3.0,NaN,NaN
7,NaN,NaN,d,2.0


In [60]:
left=pd.DataFrame({'key1':['foo','foo','bar']
                  ,'key2':['one','two','one']
                  ,'lval':[1,2,3]})

In [61]:
right=pd.DataFrame({'key3':['foo','foo','bar','bar']
                   ,'key4':['one','one','one','two']
                   ,'rval':[4,5,6,7]})

In [62]:
right

,key3,key4,rval
0,foo,one,4
1,foo,one,5
2,bar,one,6
3,bar,two,7


In [65]:
pd.merge(left,right,left_on=['key1','key2'],right_on=['key3','key4'],how='outer')

,key1,key2,lval,key3,key4,rval
0,foo,one,1.0,foo,one,4.0
1,foo,one,1.0,foo,one,5.0
2,foo,two,2.0,NaN,NaN,NaN
3,bar,one,3.0,bar,one,6.0
4,NaN,NaN,NaN,bar,two,7.0


### 索引上的合并

In [66]:
left1=pd.DataFrame({'key':list('abaabc')
                   ,'value':range(6)})

In [67]:
right1=pd.DataFrame({'group_val':[3.5,7]}
                   ,index=['a','b'])

In [68]:
left1

,key,value
0,a,0
1,b,1
2,a,2
3,a,3
4,b,4
5,c,5


In [69]:
left2

NameError: name 'left2' is not defined

In [70]:
right1

,group_val
a,3.5
b,7.0


In [71]:
pd.merge(left1,right1,left_on='key',right_index=True)

,key,value,group_val
0,a,0,3.5
2,a,2,3.5
3,a,3,3.5
1,b,1,7.0
4,b,4,7.0


In [72]:
lefth=pd.DataFrame({'key1':['Ohio','Ohio','Ohio','Nevada','Nevada']
                   ,'key2':[2000,2001,2002,2001,2002]
                   ,'data':np.arange(5.)})

In [73]:
lefth

,key1,key2,data
0,Ohio,2000,0.0
1,Ohio,2001,1.0
2,Ohio,2002,2.0
3,Nevada,2001,3.0
4,Nevada,2002,4.0


In [74]:
righth=pd.DataFrame(np.arange(12).reshape((6,2))
                   ,index=[['Nevada','Nevada','Ohio','Ohio','Ohio','Ohio']
                          ,[2001,2000,2000,2000,2001,2002]]
                   ,columns=['event1','event2'])

In [75]:
lefth

,key1,key2,data
0,Ohio,2000,0.0
1,Ohio,2001,1.0
2,Ohio,2002,2.0
3,Nevada,2001,3.0
4,Nevada,2002,4.0


In [76]:
righth

event1  event2
Nevada 2001       0       1
       2000       2       3
Ohio   2000       4       5
       2000       6       7
       2001       8       9
       2002      10      11

In [77]:
pd.merge(lefth,righth,left_on=['key1','key2'],right_index=True)

,key1,key2,data,event1,event2
0,Ohio,2000,0.0,4,5
0,Ohio,2000,0.0,6,7
1,Ohio,2001,1.0,8,9
2,Ohio,2002,2.0,10,11
3,Nevada,2001,3.0,0,1


In [81]:
left2=pd.DataFrame(np.arange(1.,7.).reshape((3,2))
                  ,index=list('ace')
                  ,columns=['Ohio','Nevada'])

In [82]:
left2

,Ohio,Nevada
a,1.0,2.0
c,3.0,4.0
e,5.0,6.0


In [83]:
right2=pd.DataFrame(np.arange(7.,15.).reshape((4,2))
                   ,index=list('bcde')
                   ,columns=['Missouri','Alabama'])

In [84]:
right2

,Missouri,Alabama
b,7.0,8.0
c,9.0,10.0
d,11.0,12.0
e,13.0,14.0


In [85]:
pd.merge(left2,right2,how='outer',left_index=True,right_index=True)

,Ohio,Nevada,Missouri,Alabama
a,1.0,2.0,NaN,NaN
b,NaN,NaN,7.0,8.0
c,3.0,4.0,9.0,10.0
d,NaN,NaN,11.0,12.0
e,5.0,6.0,13.0,14.0


In [86]:
left2.join(right2,how='outer')

,Ohio,Nevada,Missouri,Alabama
a,1.0,2.0,NaN,NaN
b,NaN,NaN,7.0,8.0
c,3.0,4.0,9.0,10.0
d,NaN,NaN,11.0,12.0
e,5.0,6.0,13.0,14.0


In [87]:
left1

,key,value
0,a,0
1,b,1
2,a,2
3,a,3
4,b,4
5,c,5


In [88]:
right1

,group_val
a,3.5
b,7.0


In [89]:
left1.join(right1,on='key')

,key,value,group_val
0,a,0,3.5
1,b,1,7.0
2,a,2,3.5
3,a,3,3.5
4,b,4,7.0
5,c,5,NaN


In [91]:
another=pd.DataFrame(np.arange(7.,15.).reshape((4,2))
                    ,index=list('acef')
                    ,columns=['New York','Oregon'])

In [92]:
another

,New York,Oregon
a,7.0,8.0
c,9.0,10.0
e,11.0,12.0
f,13.0,14.0


In [93]:
[right2,another]

[   Missouri  Alabama
 b       7.0      8.0
 c       9.0     10.0
 d      11.0     12.0
 e      13.0     14.0,    New York  Oregon
 a       7.0     8.0
 c       9.0    10.0
 e      11.0    12.0
 f      13.0    14.0]

In [94]:
right2

,Missouri,Alabama
b,7.0,8.0
c,9.0,10.0
d,11.0,12.0
e,13.0,14.0


In [95]:
left2

,Ohio,Nevada
a,1.0,2.0
c,3.0,4.0
e,5.0,6.0


In [96]:
left2.join([right2,another])

,Ohio,Nevada,Missouri,Alabama,New York,Oregon
a,1.0,2.0,NaN,NaN,7.0,8.0
c,3.0,4.0,9.0,10.0,9.0,10.0
e,5.0,6.0,13.0,14.0,11.0,12.0


### 轴向连接

In [97]:
arr=np.arange(12).reshape((3,4))

In [98]:
arr

array([[ 0,  1,  2,  3],
       [ 4,  5,  6,  7],
       [ 8,  9, 10, 11]])

In [101]:
np.concatenate([arr,arr],1)

array([[ 0,  1,  2,  3,  0,  1,  2,  3],
       [ 4,  5,  6,  7,  4,  5,  6,  7],
       [ 8,  9, 10, 11,  8,  9, 10, 11]])

In [106]:
type(arr)

numpy.ndarray

In [107]:
s1=pd.Series([0,1],index=['a','b'])

In [108]:
s2=pd.Series([2,3,4],index=list('cde'))

In [109]:
s3=pd.Series([5,6],index=list('fg'))

In [111]:
pd.concat([s1,s2,s3],1)

C:\ProgramData\Anaconda3\lib\site-packages\ipykernel_launcher.py:1: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=False'.

To retain the current behavior and silence the warning, pass 'sort=True'.

  """Entry point for launching an IPython kernel.


,0,1,2
a,0.0,NaN,NaN
b,1.0,NaN,NaN
c,NaN,2.0,NaN
d,NaN,3.0,NaN
e,NaN,4.0,NaN
f,NaN,NaN,5.0
g,NaN,NaN,6.0


In [112]:
s4=pd.concat([s1,s3])

In [113]:
s4

a    0
b    1
f    5
g    6
dtype: int64

In [114]:
pd.concat([s1,s4],1)

C:\ProgramData\Anaconda3\lib\site-packages\ipykernel_launcher.py:1: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=False'.

To retain the current behavior and silence the warning, pass 'sort=True'.

  """Entry point for launching an IPython kernel.


,0,1
a,0.0,0
b,1.0,1
f,NaN,5
g,NaN,6


In [120]:
pd.concat([s1,s4],1,'inner')

,0,1
a,0,0
b,1,1


In [121]:
pd.concat([s1,s4],1,join_axes=[list('acbe')])

,0,1
a,0.0,0.0
c,NaN,NaN
b,1.0,1.0
e,NaN,NaN


In [122]:
result=pd.concat([s1,s1,s3]
                ,keys=['one','twp','three'])

In [123]:
result

one    a    0
       b    1
twp    a    0
       b    1
three  f    5
       g    6
dtype: int64

In [124]:
result.unstack()

,a,b,f,g
one,0.0,1.0,NaN,NaN
twp,0.0,1.0,NaN,NaN
three,NaN,NaN,5.0,6.0
